# Práctica 6: *Fine-tuning en producción*

**Fecha de entrega: 11 de Mayo de 2025**

In [ ]:
#!jupytext --set-formats ipynb,py 06_finetuning_en_produccion.ipynb

[jupytext] Reading 06_finetuning_en_produccion.ipynb in format ipynb
[jupytext] Updating notebook metadata with '{"jupytext": {"formats": "ipynb,py"}}'
[jupytext] Updating 06_finetuning_en_produccion.ipynb
[jupytext] Updating 06_finetuning_en_produccion.py


In [ ]:
# Bibliotecas
import pandas as pd
import numpy as np
from os.path import join
from transformers import Trainer, AutoModelForSequenceClassification, AutoTokenizerñ
from transformers import DataCollatorWithPadding
import torch
import os

import utils


# Descargadas
from codecarbon import EmissionsTracker
import gradio as gr

## Hacer el fine tuning del modelo


- Selecciona un modelo pre-entrenado como base y realiza *fine-tuning* para resolver alguna tarea de NLP que te parezca reelevante
  - Procura utilizar datasets pequeños para que sea viable
  - Recuerda las posibles tareas disponibles en HF `*For<task>`

In [4]:
# Detectar el dispositivo (CPU o GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
# rutas con los datos
main_dir = "./data"
train_rute = main_dir + '/train_es_LimpiezaFinal.csv'
test_rute = main_dir + '/dev_es_LimpiezaFinal.csv'

# Nombre del modelo
model_name = "delarosajav95/HateSpeech-BETO-cased-v2"
# Nombre del tokenizador
tokenizer_name = "delarosajav95/HateSpeech-BETO-cased-v2"

# Definir el directorio de salida con una marca de tiempo
run_dir = join('./Ejecuciones', 'runs', f'Model_{model_name.replace("/", "_")}', utils.timestamp())

# Cargar los datos y procesarlos
train_dataset,eval_dataset,test_dataset = utils.prepareDatasets(train_rute, test_rute)

# Tokenizar los datos 
tokenized_train, tokenized_dev, tokenizer = utils.tokenizer(tokenizer_name,
                                                                train_dataset,
                                                                eval_dataset)

# Preparar el objeto para la recolección y padding dinámico de los datos
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Inicializar el modelo pre-entrenado
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

# Generar los argumentos de entrenamiento
training_args= utils.newTrainingArguments(run_dir)

# Configurar el entrenador (Trainer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics= utils.compute_metrics
)

# Iniciar el rastreador
tracker = EmissionsTracker()
tracker.start()

# Entrenar el modelo
trainer.train()
print(f"Entrenamiento del modelo {model_name} terminada")

# Evaluar el modelo en el conjunto de prueba
evaluation_results = trainer.evaluate()

print(f"Evaluacion del modelo {model_name} terminada")
print(evaluation_results)

# Guardar la informacion del entrenamiento y la evaluacion
utils.save_info(model,run_dir,trainer,training_args,tokenizer)

# Detener y mostrar emisiones
emissions = tracker.stop()
print(f"Emisiones estimadas durante el ajuste fino del modelo: {emissions:.6f} kg CO₂")

Map: 100%|██████████| 368/368 [00:00<00:00, 28619.98 examples/s]


Tokenizacion del modelo terminada


/home/dianasalgado/.local/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_61546/1371394887.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
[codecarbon WARNING @ 13:20:51] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 13:20:51] [setup] RAM Tracking...
[codecarbon INFO @ 13:20:51] [setup] CPU Tracking...
[codecarbon WARNING @ 13:20:52] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 13:20:52] CPU Model on constant consumption mode: Intel(R) Core(TM) i9-10850K CPU @ 3.60GHz
[codecarbon WARNING @ 13

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.631200,0.449598,0.826087,0.827957
2,0.329200,0.758325,0.834239,0.839895
3,0.146500,0.876154,0.831522,0.841837


[codecarbon INFO @ 13:21:15] Energy consumed for RAM : 0.000086 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 13:21:16] Delta energy consumed for CPU with cpu_load : 0.000058 kWh, power : 13.391666666666667 W
[codecarbon INFO @ 13:21:16] Energy consumed for All CPU : 0.000058 kWh
[codecarbon INFO @ 13:21:16] Energy consumed for all GPUs : 0.001120 kWh. Total GPU Power : 252.0076046426905 W
[codecarbon INFO @ 13:21:16] 0.001264 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:21:16] Energy consumed for RAM : 0.000086 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 13:21:16] Delta energy consumed for CPU with cpu_load : 0.000057 kWh, power : 13.191666666666668 W
[codecarbon INFO @ 13:21:16] Energy consumed for All CPU : 0.000057 kWh
[codecarbon INFO @ 13:21:16] Energy consumed for all GPUs : 0.001150 kWh. Total GPU Power : 258.5313336662824 W
[codecarbon INFO @ 13:21:16] 0.001292 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:21:30] Energy consumed for RAM :

Entrenamiento del modelo delarosajav95/HateSpeech-BETO-cased-v2 terminada


[codecarbon INFO @ 13:22:45] Energy consumed for RAM : 0.000569 kWh. RAM Power : 20.0 W


Evaluacion del modelo delarosajav95/HateSpeech-BETO-cased-v2 terminada
{'eval_loss': 0.4495979845523834, 'eval_accuracy': 0.8260869565217391, 'eval_f1': 0.8279569892473119, 'eval_runtime': 2.5274, 'eval_samples_per_second': 145.603, 'eval_steps_per_second': 36.401, 'epoch': 3.0}


[codecarbon INFO @ 13:22:46] Delta energy consumed for CPU with cpu_load : 0.000051 kWh, power : 12.552083333333334 W
[codecarbon INFO @ 13:22:46] Energy consumed for All CPU : 0.000383 kWh
[codecarbon INFO @ 13:22:46] Energy consumed for all GPUs : 0.007348 kWh. Total GPU Power : 156.24832849115776 W
[codecarbon INFO @ 13:22:46] 0.008301 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:22:49] Energy consumed for RAM : 0.000585 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 13:22:49] Delta energy consumed for CPU with cpu_load : 0.000010 kWh, power : 12.5 W
[codecarbon INFO @ 13:22:49] Energy consumed for All CPU : 0.000393 kWh
[codecarbon INFO @ 13:22:49] Energy consumed for all GPUs : 0.007480 kWh. Total GPU Power : 139.47758824241726 W
[codecarbon INFO @ 13:22:49] 0.008458 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:22:49] 0.039214 g.CO2eq/s mean an estimation of 1,236.661527426431 kg.CO2eq/year


Emisiones estimadas durante el ajuste fino del modelo: 0.004290 kg CO₂


## Poner en producción el modelo

Este codigo se encuentra en la carpeta *detectorSexismo*

El url de la aplicación es : https://huggingface.co/spaces/diana-salgado/detectorSexismo 

## Reporte de la actividad

Durante esta actividad, desarrollé y publiqué una aplicación web basada en un modelo de lenguaje entrenado para clasificar textos en español como sexistas o no sexistas. La tarea se resolvió de forma satisfactoria, ya que el modelo logra ofrecer resultados interpretables y rápidos para cualquier oración corta escrita en español, lo cual resulta útil tanto en contextos académicos como en análisis social o monitoreo de contenido en redes.

**Utilidad de la Aplicación**

La app es particularmente útil para usuarios que desean identificar sesgos de género en lenguaje cotidiano, especialmente en publicaciones de redes sociales como Twitter. Al estar disponible públicamente en Hugging Face Spaces, es accesible desde cualquier dispositivo y puede servir como herramienta educativa o de apoyo en proyectos de investigación relacionados con el análisis de discurso o igualdad de género.

**Fine-tuning del Modelo**

El entrenamiento del modelo fue relativamente sencillo, ya que reutilicé código previamente desarrollado durante mi servicio social. Originalmente, probé múltiples modelos antes de seleccionar el más efectivo para español: [delarosajav95/HateSpeech-BETO-cased-v2]. El conjunto de datos utilizado fue el corpus de la competencia EXIST 2024, compuesto por tweets en español, al cual se le aplicó preprocesamiento (limpieza de símbolos no alfanuméricos, stopwords, conversión a minúsculas, etc.).

Dado que entrenar en mi laptop con CPU resultaba demasiado lento, realicé el ajuste fino en una computadora del laboratorio LATTE del instituto, equipada con una NVIDIA GeForce RTX 3080, logrando completar el entrenamiento en menos de 5 minutos.

**Producción y Despliegue**

Poner el modelo en producción representó un mayor reto, ya que era la primera vez que lo publicaba en Hugging Face Spaces. Aunque inicialmente fue una curva de aprendizaje considerable, el proceso fue bastante amigable gracias a la documentación oficial de Hugging Face y al apoyo recibido de herramientas como ChatGPT. Aprendí a empaquetar correctamente el modelo, generar el archivo requirements.txt, y estructurar el código para que fuese compatible con Gradio.

## Extra

**Reporte de emisiones con CodeCarbon**

Utilicé CodeCarbon para medir el impacto ambiental del ajuste fino de mi modelo. El entrenamiento fue realizado en una máquina con GPU NVIDIA GeForce RTX 3080, y las emisiones estimadas fueron de **0.0041 kg de CO₂**, lo que equivale aproximadamente a la energía consumida por una lámpara LED encendida durante unas 3 horas. No fue posible medir las emisiones de la aplicación en producción, ya que Hugging Face Spaces no permite el monitoreo del consumo energético del entorno, por lo que CodeCarbon no puede ser utilizado de forma efectiva en ese contexto.